# 교차 검증과 그리드 서치
## 검증 세트

In [2]:
import pandas as pd # pandas 라이브러리를 불러옵니다.

wine = pd.read_csv('https://raw.githubusercontent.com/rickiepark/hg-mldl/master/wine.csv') # 와인 데이터를 불러옵니다.

In [3]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy() # 'alcohol', 'sugar', 'pH' 열을 NumPy 배열로 변환합니다.
target = wine['class'].to_numpy() # 'class' 열을 NumPy 배열로 변환합니다.

In [4]:
from sklearn.model_selection import train_test_split # train_test_split 함수를 불러옵니다.

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [5]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [6]:
print(sub_input.shape, val_input.shape) # (415, 3) (104, 3) - 서브 훈련 세트와 검증 세트의 크기를 출력합니다.

(4157, 3) (1040, 3)


In [7]:
from sklearn.tree import DecisionTreeClassifier # 결정 트리 분류기를 불러옵니다.

dt = DecisionTreeClassifier(random_state=42)    # 결정 트리 분류기 객체를 생성합니다.
dt.fit(sub_input, sub_target)   # 서브 훈련 세트로 모델을 학습시킵니다.

print(dt.score(sub_input, sub_target)) # 서브 훈련 세트에 대한 정확도를 출력합니다.
print(dt.score(val_input, val_target)) # 검증 세트에 대한 정확도를 출력합니다.

0.9971133028626413
0.864423076923077


## 교차 검증

In [8]:
from sklearn.model_selection import cross_validate # 교차 검증을 위한 함수를 불러옵니다.

scores = cross_validate(dt, train_input, train_target) # 교차 검증을 수행합니다.
print(scores) # {'fit_time': array([0.00299978, 0.00200033, 0.00200033, 0.00200033, 0.00200033]),
# 'score_time': array([0.00100017, 0.00100017, 0.00100017, 0.00100017, 0.00100017]),
# 'test_score': array([0.88888889, 0.86111111, 0.86111111, 0.86111111, 0.88888889])}

{'fit_time': array([0.01107335, 0.006001  , 0.00701594, 0.00706387, 0.00608373]), 'score_time': array([0.00099659, 0.001508  , 0.00100374, 0.00100207, 0.00096512]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [9]:
import numpy as np # NumPy 라이브러리를 불러옵니다.

print(np.mean(scores['test_score'])) # 0.8722222222222222 - 교차 검증의 평균 정확도를 출력합니다.

0.855300214703487


In [10]:
from sklearn.model_selection import StratifiedKFold # StratifiedKFold 클래스를 불러옵니다.

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold()) # StratifiedKFold를 사용하여 교차 검증을 수행합니다.
print(np.mean(scores['test_score'])) # 0.8722222222222222 - StratifiedKFold를 사용한 교차 검증의 평균 정확도를 출력합니다.

0.855300214703487


In [11]:
from sklearn.model_selection import StratifiedKFold # StratifiedKFold 클래스를 불러옵니다.

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) # StratifiedKFold 객체를 생성합니다.

scores = cross_validate(dt, train_input, train_target, cv=splitter)     # StratifiedKFold를 사용하여 교차 검증을 수행합니다.
print(np.mean(scores['test_score'])) # 0.8722222222222222 - StratifiedKFold를 사용한 교차 검증의 평균 정확도를 출력합니다.

0.8574181117533719


## 하이퍼파라미터 튜닝

In [12]:
from sklearn.model_selection import GridSearchCV # GridSearchCV 클래스를 불러옵니다.

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}  # 하이퍼파라미터 그리드를 정의합니다.

In [13]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)  # GridSearchCV 객체를 생성합니다.

In [14]:
gs.fit(train_input, train_target) # 훈련 세트로 모델을 학습시킵니다.

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [15]:
dt = gs.best_estimator_ # 가장 좋은 모델 출력
print(dt.score(train_input, train_target)) # 훈련 세트 점수

0.9615162593804117


In [16]:
print(gs.cv_results_['mean_test_score']) # 교차 검증 점수 확인

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [17]:
print(gs.best_params_)  # 최적 하이퍼파라미터

{'min_impurity_decrease': 0.0001}


In [18]:
print(gs.cv_results_['mean_test_score']) # 교차 검증 점수 확인

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [19]:
best_index = np.argmax(gs.cv_results_['mean_test_score']) # 가장 좋은 인덱스 찾기
print(gs.cv_results_['params'][best_index]) # 가장 좋은 파라미터 출력

{'min_impurity_decrease': 0.0001}


In [20]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [21]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [22]:
print(gs.best_params_)  # 가장 좋은 파라미터 출력

{'max_depth': 14, 'min_impurity_decrease': np.float64(0.0004), 'min_samples_split': 12}


In [23]:
print(np.max(gs.cv_results_['mean_test_score'])) # 가장 좋은 교차 검증 점수 출력

0.8683865773302731


## 랜덤 서치

In [24]:
from scipy.stats import uniform, randint # uniform과 randint를 불러옵니다.

In [25]:
rgen = randint(0, 10) # 0부터 10까지의 정수 중에서 무작위로 선택하는 랜덤 정수 생성기입니다.
rgen.rvs(10) # 10개의 무작위 정수 샘플을 생성합니다.

array([1, 0, 8, 4, 3, 2, 8, 2, 4, 7])

In [26]:
np.unique(rgen.rvs(1000), return_counts=True) 
# 0부터 10까지의 정수 중에서 무작위로 선택한 1000개의 샘플의 고유 값과 그 개수를 출력합니다.

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([104,  86, 103, 103,  89, 106,  99, 100, 104, 106]))

In [27]:
ugen = uniform(0, 1) # 0부터 1까지의 실수 중에서 무작위로 선택하는 랜덤 실수 생성기입니다.
ugen.rvs(10) # 10개의 무작위 실수 샘플을 생성합니다.

array([0.94883431, 0.42714817, 0.25540998, 0.24010057, 0.70516202,
       0.54169873, 0.82322474, 0.95297462, 0.28472952, 0.89472594])

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),  # 0.0001부터 0.001까지의 실수 범위에서 무작위로 선택
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV # RandomizedSearchCV 클래스를 불러옵니다.

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target) # 훈련 세트로 모델을 학습시킵니다.

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000016EEF64F310>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000016EEF64FE90>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000016EEF64D090>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000016EEF64EE10>},
                   random_state=42)

In [ ]:
print(gs.best_params_)  # 가장 좋은 파라미터 출력

{'max_depth': 39, 'min_impurity_decrease': np.float64(0.00034102546602601173), 'min_samples_leaf': 7, 'min_samples_split': 13}


In [31]:
print(np.max(gs.cv_results_['mean_test_score'])) # 가장 좋은 교차 검증 점수 출력

0.8695428296438884


In [ ]:
dt = gs.best_estimator_ # 가장 좋은 모델을 가져옵니다.

print(dt.score(test_input, test_target)) # 테스트 세트에 대한 정확도를 출력합니다.

0.86


## 확인 문제

In [ ]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target) # 훈련 세트로 모델을 학습시킵니다.

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000016EEF64F310>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000016EEF64FE90>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000016EEF64D090>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000016EEF64EE10>},
                   random_state=42)

In [ ]:
print(gs.best_params_) # 가장 좋은 파라미터 출력
print(np.max(gs.cv_results_['mean_test_score'])) # 가장 좋은 교차 검증 점수 출력

dt = gs.best_estimator_ # 가장 좋은 모델을 가져옵니다.
print(dt.score(test_input, test_target)) # 테스트 세트에 대한 정확도를 출력합니다.

{'max_depth': 43, 'min_impurity_decrease': np.float64(0.00011407982271508446), 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
